In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 迁移：使用预构建的训练容器进行自定义图像分类

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-image-classification-prebuilt-container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-image-classification-prebuilt-container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-custom-image-classification-prebuilt-container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用 Python 版本的 Vertex AI SDK，在使用预构建容器进行训练的同时，部署一个用于在线和批量预测的自定义图像分类模型。

了解更多关于[迁移到 Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的信息。

### 目标

在本教程中，您将学习如何使用预构建的容器和 Vertex AI 训练来训练一个 tensorflow 图像分类模型。训练完成后，您还将使用预构建的容器将模型部署到 Vertex AI，并对其进行批量和在线预测。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 训练
- Vertex AI 模型注册表
- Vertex AI 预测
- Vertex AI 批量预测
- Vertex AI 终端

执行的步骤包括：

- *将训练代码打包成 python 应用程序。*
- *使用 Cloud Build 和 Artifact Registry 将训练应用程序容器化。*
- *在 Vertex AI 中创建一个自定义容器训练作业并运行它。*
- *评估从训练作业生成的模型。*
- *在 Vertex AI 模型注册表为训练的模型创建一个模型资源。*
- *运行 Vertex AI 的批量预测作业。*
- *将模型资源部署到 Vertex AI 终端。*
- *在模型资源上运行在线预测作业。*
- *清理创建的资源。*

数据集

本教程使用的数据集是[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经集成到TensorFlow中。训练好的模型会预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

费用

本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 安装

安装执行此笔记本所需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform google-cloud-storage

! apt-get update && apt-get install -y python3-opencv-headless
! apt-get install -y libgl1-mesa-dev
! pip3 install --upgrade opencv-python-headless  
! pip3 install --upgrade tensorflow  

### 仅限协作：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

###验证您的Google Cloud账号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶以存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

1. 设置变量

接下来，设置一些在教程中使用的变量。
2. 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的顶点SDK

为您的项目和相应的存储桶初始化Python的顶点AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 的 GPU 容器映像分配给每个 VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定 `(None, None)` 来在 CPU 上运行一个容器映像。

在此处了解更多关于您地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

*注意*：TF 2.3 之前的 GPU 支持版本将无法加载此教程中的自定义模型。这是一个已知问题，在 TF 2.3 中得到解决 —— 这是由在服务函数中生成的静态图操作引起的。如果您在您自己的自定义模型上遇到此问题，请使用带有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器镜像。

要查看最新列表，请访问[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

要查看最新列表，请访问[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2-5"

if TRAIN_GPU:
    TRAIN_VERSION = "tf-gpu.{}".format(TF)
else:
    TRAIN_VERSION = "tf-cpu.{}".format(TF)
if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下情况不支持训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训作业组装 Python 包。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是安装包到 Docker 镜像的操作环境的说明。

文件 `trainer/task.py` 是执行自定义培训作业的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们用点 (`trainer.task`) 替换目录斜杠，并且去掉文件后缀（`.py`）。

#### 包组装

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我们不会详细展开，只是让您浏览一下。总结如下：

- 从命令行获取保存模型结果的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的.tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

训练一个模型

### [training.create-python-pre-built-container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)

### [建立 Python 预构建容器](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行这个任务。

#### 创建自定义训练任务

通过使用`CustomTrainingJob`类创建一个自定义训练任务，该类有以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：用于训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="cifar10_" + UUID,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

*示例输出：*

<google.cloud.aiplatform.training_jobs.CustomTrainingJob object at 0x7feab1346710> 

*示例输出：*

<google.cloud.aiplatform.training_jobs.CustomTrainingJob object at 0x7feab1346710>

#### 运行定制训练作业

接下来，您可以通过调用`run`方法来运行定制作业，从而开始训练作业，参数如下：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器副本的加速器数量。
- `base_output_dir`：将模型工件写入的云存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

评估模型

加载保存的模型

您的模型以TensorFlow SavedModel格式存储在云存储桶中。现在从云存储桶加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载，您可以使用TF.Keras的model.load_model()方法，将保存模型的云存储路径传递给它，该路径由MODEL_DIR指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`中加载CIFAR10测试（留存）数据，使用方法`load_data()`。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是包含两个元素的元组：图像数据和相应的标签。

您不需要训练数据，因此将其加载为`(_,_)`。

在可以进行评估之前，您需要预处理数据：

`x_test`：
1. 通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这将用介于0和1之间的32位浮点数替换每个单字节整数像素。

`y_test`：<br/>
2. 标签目前是标量（稀疏）。如果您回顾一下`trainer/task.py`脚本中的`compile()`步骤，您会发现它是为稀疏标签编译的。因此我们不需要做其他处理。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估定制作业中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

### [general.import-model](https://cloud.google.com/vertex-ai/docs/general/import-model)
### [general.import-model](https://cloud.google.com/vertex-ai/docs/general/import-model)

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩（例如 JPEG）的图像字节编码为 base 64，这样可以使内容在通过网络传输二进制数据时安全无法修改。由于部署的模型期望输入数据为原始（未压缩）字节，因此您需要确保将 base 64 编码的数据转换回原始字节，然后将其作为输入传递给部署的模型。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型上作为预处理步骤。添加一个 `@tf.function` 装饰器，使服务函数与底层模型融合（而不是上游在 CPU 上）。

当您发送预测或解释请求时，请求的内容将被 base 64 解码为一个 Tensorflow 字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将 `tf.string` 预处理为原始（未压缩）的 numpy 字节（`preprocess_fn`）以满足模型的输入要求：
- `io.decode_jpeg` - 解压缩 JPG 图像，返回一个具有三个通道（RGB）的 Tensorflow 张量。
- `image.convert_image_dtype` - 将整数像素值转换为浮点数 32。
- `image.resize` - 调整图像大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（归一化）在 0 到 1 之间。

此时，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务功能的签名。为什么呢？当我们将数据发送用作 HTTP 请求数据包进行预测时，图像数据是 base64 编码的，而我们的 TF.Keras 模型需要 numpy 输入。您的服务功能将会将数据从 base64 转换为 numpy 数组。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 - 这将在您进行预测请求时稍后使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到一个`Model`资源，使用以下参数：

- `display_name`：`Model`资源的可读名称。
- `artifact`：训练模型工件的存储位置。
- `serving_container_image_uri`：提供容器图像。
- `sync`：是否异步或同步执行上传。

如果`upload()`方法是异步运行的，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aip.Model.upload(
    display_name="cifar10_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

示例输出：

INFO:google.cloud.aiplatform.models:创建模型
INFO:google.cloud.aiplatform.models:创建模型后端LRO：projects/759209241365/locations/us-central1/models/925164267982815232/operations/3458372263047331840
INFO:google.cloud.aiplatform.models:模型已创建。资源名称：projects/759209241365/locations/us-central1/models/925164267982815232
INFO:google.cloud.aiplatform.models:在另一个会话中使用此模型：
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/759209241365/locations/us-central1/models/925164267982815232')

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions) 

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

获取测试项目

您将使用数据集的测试（保留）部分中的示例作为测试项目。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

### 准备请求内容
您将发送CIFAR10图像作为压缩的JPG图像，而不是原始未压缩的字节： 

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘作为压缩的JPEG图像。
- 将图像数据从\[0,1)范围反归一化到[0,255)。
- 将32位浮点值转换为8位无符号整数。

In [ ]:
import cv2

cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_URI/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_URI/tmp2.jpg

test_item_1 = BUCKET_URI + "/tmp1.jpg"
test_item_2 = BUCKET_URI + "/tmp2.jpg"

### 制作批量输入文件

现在制作一个批量输入文件，你将把它存储在你的本地云存储桶中。批量输入文件只能是 JSONL 格式。对于 JSONL 文件，你需要为每个数据项（实例）每行制作一个字典条目。字典包含以下键/值对：

- `input_name`：底层模型的输入层的名称。
- `'b64'`：指示内容为 base64 编码的键。
- `content`：压缩的 JPG 图像字节，作为 base64 编码的字符串。

预测请求中的每个实例都是以下形式的字典条目：

{ serving_input: {'b64': content}}

为了将图像数据传递给预测服务，你需要将字节编码为 base64 -- 这样在通过网络传输二进制数据时，可以确保内容不会被修改。

- `tf.io.read_file`：将压缩的 JPG 图像按原始字节读入内存。
- `base64.b64encode`：将原始字节编码为 base64 编码的字符串。

In [ ]:
import base64
import json

gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict（）方法来进行批量预测，具有以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `sync`：如果设置为True，调用将会阻塞，同时等待异步批处理作业的完成。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="cifar10_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

示例输出：

INFO: google.cloud.aiplatform.jobs: 创建 BatchPredictionJob
<google.cloud.aiplatform.jobs.BatchPredictionJob 对象在 0x7f806a6112d0 处等待上游依赖完成。
INFO: google.cloud.aiplatform.jobs: 创建了 BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
INFO: google.cloud.aiplatform.jobs: 要在另一个会话中使用此 BatchPredictionJob：
INFO: google.cloud.aiplatform.jobs: bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
INFO: google.cloud.aiplatform.jobs: 查看批量预测任务：
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
INFO: google.cloud.aiplatform.jobs: BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：JobState.JOB_STATE_RUNNING

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻止直到批量预测作业完成为止。

In [ ]:
batch_predict_job.wait()

示例输出：

INFO:google.cloud.aiplatform.jobs:已创建批量预测作业。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
INFO:google.cloud.aiplatform.jobs:要在另一个会话中使用此批量预测作业：
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
INFO:google.cloud.aiplatform.jobs:查看批量预测作业：
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:批量预测作业 projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:已完成批量预测作业运行。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果会被写入到你在批量预测请求中指定的云存储输出存储桶中。你可以调用方法iter_outputs()来获取包含结果的每个云存储文件的列表。每个文件都以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

{'instance': {'bytes_inputs': {'b64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKC...'}}, 'prediction': [0.0560616329, 0.122713037, 0.121289924, 0.109751239, 0.121320881, 0.0897410363, 0.145011798, 0.0976110101, 0.0394041203, 0.0970953554]}

进行在线预测

### [predictions.deploy-model-api]（https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api） 

### [预测部署模型 API]（https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api）

## 部署模型

接下来，将您的模型部署用于在线预测。要部署模型，您需要调用`deploy`方法，并指定以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：流量分配在端点上流向该模型的百分比，指定为一个或多个键/值对的字典。
如果只有一个模型，那么指定为{ "0": 100 }，其中"0"指代上传的这个模型，而100表示100%的流量。
如果端点上已存在其他模型，需要将流量分配给它们，则使用model_id指定为{ "0": 百分比, model_id: 百分比, ... }，其中model_id是已部署到端点的现有模型的模型id。各百分比必须相加为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作实例的加速器数量。
- `starting_replica_count`：最初规定的计算实例数量。
- `max_replica_count`：可扩展到的最大计算实例数量。在本教程中，仅会提供一个实例。

In [ ]:
DEPLOYED_NAME = "cifar10-" + UUID

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

示例输出：

    信息：google.cloud.aiplatform.models：创建端点
    信息：google.cloud.aiplatform.models：创建端点支持的 LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    信息：google.cloud.aiplatform.models：端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models：在另一个会话中使用此端点：
    信息：google.cloud.aiplatform.models：endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    信息：google.cloud.aiplatform.models：将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models：部署端点模型支持的 LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    信息：google.cloud.aiplatform.models：端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

获取测试项目

您将使用数据集的测试（保留）部分中的一个示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

###准备请求内容
您将以压缩后的JPG图像的形式发送CIFAR10图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘，以压缩后的JPEG图像形式。
  - 将图像数据从\[0,1)范围反归一化为[0,255)。
  - 将32位浮点值转换为8位无符号整数。
- `tf.io.read_file`：以原始字节的形式将压缩后的JPG图像读入内存。
- `base64.b64encode`：将原始字节编码为base64编码的字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源上，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

在这个示例中，您的测试项目位于一个 Cloud Storage 存储桶中，您可以使用 `tf.io.gfile.Gfile()` 打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为 base64 格式 -- 这样可以在通过网络传输二进制数据时使内容免受修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

`predict()` 调用的响应是一个 Python 字典，具有以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `predictions`: 每个类别标签的预测置信度，介于 0 和 1 之间。
- `deployed_model_id`: 执行预测的已部署 `Model` 资源的 Vertex AI 标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

*示例输出：*

    预测(predictions=[[0.0560616292, 0.122713044, 0.121289924, 0.109751239, 0.121320873, 0.0897410288, 0.145011798, 0.0976110175, 0.0394041166, 0.0970953479]], 部署模型标识='4087166195420102656', 解释=None)

## 卸载模型

当您完成预测时，您可以从`Endpoint`资源中卸载模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理本项目中使用的所有 Google Cloud 资源，您可以删除用于本教程的 Google Cloud 项目。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI